<a href="https://colab.research.google.com/github/Madaa73/Health_Costst/blob/main/Copia_de_fcc_predict_health_costs_with_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import libraries. You may or may not use all of these.
!pip install -q git+https://github.com/tensorflow/docs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers,models, regularizers

import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling

In [ ]:
# Import data
!wget https://cdn.freecodecamp.org/project-data/health-costs/insurance.csv
dataset = pd.read_csv('insurance.csv')
dataset.tail()

In [ ]:
dataset.columns = dataset.columns.str.strip()
print(dataset.columns)

In [ ]:
# Convertir datos categóricos a valores numéricos
dataset = pd.get_dummies(dataset, columns=['sex', 'smoker', 'region'], drop_first=True)

In [ ]:
train_dataset, test_dataset = train_test_split(dataset, test_size=0.2, random_state=42)
print(dataset.columns)

In [ ]:
# Verifica las columnas del dataset
print(dataset.columns)
# Las etiquetas son solo la columna 'expenses'
train_labels = train_dataset.pop('expenses')  # Extraer 'expenses' como etiquetas para entrenamiento
test_labels = test_dataset.pop('expenses')    # Lo mismo para las etiquetas de prueba

In [ ]:
# Crear el modelo
model = tf.keras.Sequential([
    layers.Dense(128, activation='relu', input_shape=[len(train_dataset.keys())], kernel_regularizer=regularizers.l2(0.01)),
    layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
    layers.Dense(64, activation='relu'),
    layers.Dense(1)
])
# Compilar el modelo
model.compile(loss=tf.keras.losses.Huber(),
              optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              metrics=['mae', 'mse'])


# Entrenar el modelo
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
history = model.fit(train_dataset, train_labels, batch_size=30, epochs=100, validation_split=0.2, verbose=1, callbacks=[early_stopping])

In [ ]:
# RUN THIS CELL TO TEST YOUR MODEL. DO NOT MODIFY CONTENTS.
# Test model by checking how well the model generalizes using the test set.
loss, mae, mse = model.evaluate(test_dataset, test_labels, verbose=2)

print("Testing set Mean Abs Error: {:5.2f} expenses".format(mae))

if mae < 3500:
  print("You passed the challenge. Great job!")
else:
  print("The Mean Abs Error must be less than 3500. Keep trying.")

# Plot predictions.
test_predictions = model.predict(test_dataset).flatten()

a = plt.axes(aspect='equal')
plt.scatter(test_labels, test_predictions)
plt.xlabel('True values (expenses)')
plt.ylabel('Predictions (expenses)')
lims = [0, 50000]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims,lims)
